In [6]:
import threading
import time
import random

In [7]:
BUFFER_SIZE = 5

# Two buffers: ping (0) and pong (1)
buffer = [[None]*BUFFER_SIZE, [None]*BUFFER_SIZE]

write_buf = 0
read_buf  = 1

data_ready = [False, False]

lock = threading.Lock()

In [8]:
def producer():
    global write_buf, read_buf

    item_id = 0
    while item_id < 10:
        with lock:
            if not data_ready[write_buf]:
                print(f"Producer writing to buffer {write_buf}")

                for i in range(BUFFER_SIZE):
                    buffer[write_buf][i] = f"item-{item_id}"
                    print(f"   produced: item-{item_id}")
                    item_id += 1
                    time.sleep(0.1)

                data_ready[write_buf] = True

                # swap buffers
                write_buf, read_buf = read_buf, write_buf

        time.sleep(0.2)

In [9]:
def consumer():
    global write_buf, read_buf

    consumed = 0
    while consumed < 10:
        with lock:
            if data_ready[read_buf]:
                print(f"Consumer reading from buffer {read_buf}")

                for item in buffer[read_buf]:
                    print("   consumed:", item)
                    consumed += 1
                    time.sleep(0.15)

                data_ready[read_buf] = False

        time.sleep(0.2)

In [10]:
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

producer_thread.start()
consumer_thread.start()

producer_thread.join()
consumer_thread.join()

print("Done")

Producer writing to buffer 0
   produced: item-0
   produced: item-1
   produced: item-2
   produced: item-3
   produced: item-4
Consumer reading from buffer 0
   consumed: item-0
   consumed: item-1
   consumed: item-2
   consumed: item-3
   consumed: item-4
Producer writing to buffer 1
   produced: item-5
   produced: item-6
   produced: item-7
   produced: item-8
   produced: item-9
Consumer reading from buffer 1
   consumed: item-5
   consumed: item-6
   consumed: item-7
   consumed: item-8
   consumed: item-9
Done
